In [1]:
from jupyter_plotly_dash import JupyterDash

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
#########
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Change for your username and password and CRUD Python module name
username = "accuser"
password = "password"
shelter = AnimalShelter("aacuser", "password")


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the line below into the app.layout code according to your design
# Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Anderson_Forestal_SNHU CS-340 Dashboard'))),
    # html.Center(html.P('Select locations base on the table for the map')),
    html.Hr(),
    html.Div(
        
# Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DRIT'},
            {'label': 'Reset - unfiltered state', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        )
    ),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Set up the features for your interactive data table to make it user-friendly for your client
# If you completed the Module Six Assignment, you can copy in the code you created here 
    editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### Add code to filter interactive data table with MongoDB queries

     if filter_type == 'WR':
        
# Determine animals distinct breed
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                              {'$or': [
                                                  {'breed': 'Labrador Retriever Mix'},          
                                                  {'breed': 'Chesa Bay Retr Mix'},
                                                  {'breed': 'Newfoundland Mix'},         
                                                  {'breed': 'Newfoundland/Labrador Retriever'},         
                                                  {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                  {'breed': 'Newfoundland/Great Pyrenees'}] 
                                               },
                                              {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                        {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                              }]
                                     })))
# Dog type and status
     elif filter_type == 'MWR':
# breeds and ages determined by Grazioso
          df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                              {'$or': [
                                                  {'breed': 'German Shepherd'},
                                                  {'breed': 'Alaskan Malamute'},
                                                  {'breed': 'Old English Sheepdog'},
                                                  {'breed': 'Rottweiler'},
                                                  {'breed': 'Siberian Husky'}]
                                               },
                                             {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},   
                                                       {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                             }]
                                     })))
# Dog type and status
     elif filter_type == 'DRIT':
#breeds and ages determined by Grazioso
          df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                              {'$or': [
                                                  {'breed': 'Doberman Pinscher'},
                                                  {'breed': 'German Shepherd'},
                                                  {'breed': 'Golden Retriever'},
                                                  {'breed': 'Bloodhound'},
                                                  {'breed': 'Rottweiler'}]
                                               },
                                             {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                        {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                             }]
                                     })))
#resets all results
     elif filter_type == 'RESET':
          df = pd.DataFrame.from_records(shelter.read({}))
    
     columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
     data=df.to_dict('records')
        
        
     return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #######
    
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()

    # Pie chart logic#
    return [
         dcc.Graph(            
             figure =  px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )

        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")
])
def update_map(viewData):
# Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
  dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    # Create some markers.  
  points = [dict(lat=55.5 + random.random(), lon=9.5 + random.random(), value=random.random()) for i in range(100)]  
  data = dlx.dicts_to_geojson(points)
  return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]



app